# Prepare kallisto outputs for differential expression analysis
See script `/cellar/users/ndefores/pipelines/20210104_rnaseq_to_gene_counts.sh` for how to run kallisto

In [ ]:
# import packages - not all will be needed so filter through based on your needs
library(data.table)
library(tidyverse)
library(RColorBrewer)
library(reshape2)
library(gdata)
library(ggrepel)
library(biomaRt)
library(fgsea)
library(tximport)
library(DESeq2)
library(gage)
library(gageData)
library(pathview)

In [ ]:
# common functions I use that I want to unmask
# masked from dplyr by S4Vectors
rename <- dplyr::rename

# masked from dplyr by AnnotationDbi
select <- dplyr::select

## Import kallisto abundance files using tximport

In [ ]:
### read in tx2gene table
# tx2gene must two columns: 1) transcript ID and 2) gene ID, and must be in this order
# the transcript ID must be the same one used in the abundance files
tx2gene <- fread("/cellar/users/ndefores/reference_genomes//hg38//gencode_GRCh38.p12_release31//gencode.v31.chr_patch_hapl_scaff.annotation.gtf_transcript_gene_table.csv",
                data.table = F)
head(tx2gene)

In [ ]:
# read in meta data table
s2c <- read.csv("/path/to/file/", stringsAsFactor=F)

## Make sure DESeq knows what is the reference using and releveling factors
# add samples as rownames for DESeq
rownames(s2c) <- s2c$sample

In [ ]:
# Read in all samples kallisto abundance files into tximport object
files <- file.path("./kallisto_runs/", s2c$sample, "abundance.h5")
names(files) <- s2c$sample
txi.kallisto <- tximport(files, type = "kallisto", ignoreAfterBar = TRUE, tx2gene = tx2gene)
lapply(txi.kallisto,class)

# Notes: txi.kallisto$abundance = TPM matrix

## Differential expression analysis by DESeq2

In [ ]:
### run DESeq for condition
# make deseq object from txi import object
dds <- DESeqDataSetFromTximport(txi.kallisto, colData = s2c, design = ~condition) 

# run DESeq for WT vs KO
start.time <- Sys.time()
dds <- DESeq(dds)
end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

In [ ]:
# Quality control - PCA
# regularized log transform count data to log2 scale for variance stabilizing effect, convert to matrix
start.time <- Sys.time()
rld <- assay(rlog(dds))
pca <- prcomp(t(rld))
summary(pca)
end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

# Plot PCA
pca.dt <- cbind(s2c, pca$x)
ggplot(pca.dt, aes(x=PC1, PC2, color = condition, label = replicate)) + geom_point() + geom_text_repel()

In [ ]:
# make DESEq results into data frame
res <- results(dds) %>%
    as.data.frame() %>%
    rownames_to_column("gene_id") %>%
    left_join(., distinct(select(tx2gene, -transcript_id)), by = "gene_id") %>%
    arrange(padj)

# get entrez IDs for genes
res$entrez <- mapIds(org.Hs.eg.db,
                     keys=as.character(res$gene), 
                     column="ENTREZID",
                     keytype="SYMBOL",
                     multiVals="first")


# add description too
res$gene_name <- mapIds(org.Hs.eg.db,
                     keys=as.character(res$gene), 
                     column="GENENAME",
                     keytype="SYMBOL",
                     multiVals="first")

head(res)

In [ ]:
## Volcano plot of DE genes
ggplot(res, aes(x = log2FoldChange, y = -log10(padj), label = gene)) + 
    geom_point() +
    geom_text_repel(data=res[1:15,], box.padding = 0.2) +
    geom_hline(yintercept = -log10(0.05), color = "red", linetype = "dashed") +
    theme_classic() +
    ylab("-log(P value)") 

## Gene set enrichment analysis
Using DESeq results `res`

In [ ]:
# Rank each gene by differential expression results
# average stat value for each gene -> rank
ranks <- res %>%
    select(gene, stat) %>%
    na.omit() %>%
    distinct() %>% 
    group_by(gene) %>% 
    summarize(stat=mean(stat)) %>%
    deframe()
head(ranks)

In [ ]:
# Example using Hallmark pathways

# Get pathways - can be downloaded from https://www.gsea-msigdb.org/gsea/msigdb/genesets.jsp
hallmark.pathways <- gmtPathways("~/annotations//gsea_pathways/h.all.v7.2.symbols.gmt")

# run fgsea
hallmark_res <- fgsea(hallmark.pathways, ranks, minSize=15, maxSize = 500, nperm=10000)
# hallmark_res_huh %>% arrange(padj)

#plot
ggplot(hallmark_res, aes(reorder(pathway, NES), NES)) +
  geom_col(aes(fill=padj<0.05)) +
  coord_flip() +
  labs(x="Pathway", y="Normalized Enrichment Score",
       title="Hallmark pathways GSEA") + 
  theme_minimal()